# Notebook Setup

In [4]:
if 'google.colab' in str(get_ipython()):
    IN_COLLAB = True
else:
    IN_COLLAB = False

if IN_COLLAB:
    #TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
    MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"
    # MY_HOME_ABS_PATH = "/content/drive/MyDrive/TFT_baseline"
    from google.colab import drive
    drive.mount('/content/drive/')
else:
    MY_HOME_ABS_PATH = "/root/co2-flux-hourly-gpp-modeling/"
    # MY_HOME_ABS_PATH = "/home/ec2-user/SageMaker/root/co2-flux-hourly-gpp-modeling"

## Import Modules

In [5]:
if IN_COLLAB:
    !pip install torch pytorch-lightning pytorch_forecasting azure-storage-blob -q
else:
    !pip install xgboost -q

In [6]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import copy
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(xgboost.__version__)

from timeit import default_timer
from datetime import datetime
import gc
import pickle
import random
from pprint import pprint

# Load locale custome modules
os.chdir(MY_HOME_ABS_PATH)
if IN_COLLAB:
     os.environ["MY_HOME_ABS_PATH"] = MY_HOME_ABS_PATH
     sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
    sys.path.append('./cred')
    sys.path.append('./code/src/tools')
    sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *
from model_pipeline_lib import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pl.seed_everything(42)

1.7.5


2023-04-15 18:42:55.230844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 18:42:55.277889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 18:42:55.278631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 18:42:56.066867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 42
Global seed set to 42


42

# Setup Experiments

## Define Local File System Constants

In [7]:
root_dir  = MY_HOME_ABS_PATH
tmp_dir   = root_dir + os.sep + '.tmp'
model_dir = root_dir + os.sep + 'data' + os.sep + 'models'
PREPRO_DIR = root_dir + "/code/src/preprocessing/preproc_objects"

container = "all-sites-data"
blob_name = "full_2010_2015_v_mvp_raw.parquet"
local_file = tmp_dir + os.sep + blob_name

## Define Features and Target

In [8]:
target_variable = 'GPP_NT_VUT_REF'

categorical_cols = ['month', 'day', 'hour', 'koppen_sub', 'koppen_main', 'MODIS_PFT', 'MODIS_IGBP', 'MODIS_LC']
realNum_cols = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 
                'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
                'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN', 'CSIF-SIFdaily',
                'PET', 'Ts', 'ESACCI-sm', 'NDWI', 'Percent_Snow', 'Fpar', 'Lai',
                'LST_Day', 'LST_Night']
features = categorical_cols + realNum_cols

supplement_cols= ['site_id', 'timestep_idx_local','gap_flag_hour','gap_flag_month',]

## Define Helper Functions


In [9]:
class X_y_set:
    X = None
    y = None

def normalize_real_cols(foldNo, realNum_cols, train_X, val_X, test_X = None):

    # Normalize data
    print(f"Normalizing real features ({len(realNum_cols)})")
    scaler = StandardScaler().fit(train_X[realNum_cols])
    train_X.loc[:,realNum_cols] = scaler.transform(train_X[realNum_cols])
    val_X.loc[:,realNum_cols] = scaler.transform(val_df[realNum_cols])

    # Save scaler object
    scaler_path = os.path.join(PREPRO_DIR, f'scaler_cv{foldNo}.joblib')
    joblib.dump(scaler, scaler_path)
    print(f"Saved scaler to {scaler_path}.")
      
    train_X.reset_index(inplace=True, drop=True)
    val_X.reset_index(inplace=True, drop=True)
    print(f"Train data size: {train_X.shape}.")
    print(f"Val data size: {val_X.shape}.")

    if test_X is not None:
        test_X.loc[:,realNum_cols] = scaler.transform(test_X[realNum_cols]) 
        test_X.reset_index(inplace=True, drop=True)
        print(f"Test data size: {test_X.shape}.") 
  
    return train_X, val_X, test_X

def split_to_X_y(df):
    dataset = X_y_set()
    dataset.X = df.drop([target_variable] + supplement_cols, axis=1)
    dataset.y = df[target_variable]                
    return dataset

def RunXGBoostCV(params, fold_data, exp_dir, debug=False, features=None):
    cv_results_df = pd.DataFrame(columns=['Fold_No', 'RMSE', 'MAE', 'R2', 'LOSS_SD', 'filename'])
    for i, cv in enumerate(fold_data):
        verbose = 0
        if debug:
            print(f"Fold {i+1}:")
            verbose = 1

        train = cv['train']
        val = cv['val']
        model = XGBRegressor(**best_params, random_state=42,
                             tree_method="approx", enable_categorical=True,
                             importance_type='gain',
                             n_jobs=-1, verbosity=verbose) # <--- Update this model!
        
        # Trim features is necessary
        if features is not None:
            train_X = train.X[features] 
            val_X = val.X[features]
        else:
            train_X = train.X
            val_X = val.X
        
        model.fit(train_X, train.y)

        # Evaluate model
        val_actuals = val.y
        val_pred = model.predict(val_X)
        rmse = np.sqrt(mean_squared_error(val_actuals, val_pred))
        mae = mean_absolute_error(val_actuals, val_pred)
        r2 = r2_score(val_actuals, val_pred)
        loss_std = np.std(val_actuals - val_pred)
        if debug:
            print(f"  Val RMSE: {rmse}, Val MAE: {mae}, Val R2/NSE: {r2}, val Loss SD: {loss_std}")

        # # Save models
        filesname = f"model_cv{i+1}.pkl"
        pickle.dump(model, open( exp_dir + os.sep + filesname, 'wb'))
        print(f"  save model to {exp_dir + os.sep + filesname}.")
        result = {'Fold_No': int(i+1), 'RMSE':rmse, 'MAE':mae, 'R2':r2 , 'LOSS_SD':loss_std, 'filename': filesname}
        cv_results_df = cv_results_df.append(result, ignore_index=True)

    print(f"Results from params {best_params}:") 
    display(cv_results_df)
    display(cv_results_df[['RMSE', 'MAE', 'R2']].mean(axis=0))

    # Save CV result of CSV
    cv_results_df.to_csv(exp_dir + os.sep + "cv_result.csv", index=False)
    print(f"CV results saved to {exp_dir + os.sep + 'cv_result.csv'}")

def feature_importance(model, model_name, columns, fig_num, fig, ax):
    # get the feature importances
    importances = pd.Series(model.feature_importances_, index=columns)#.sort_values(ascending=False)
    # Visualize
    importances_sorted = importances.sort_values(ascending=True)[-30:]

    ax[fig_num].barh(importances_sorted.index, importances_sorted.values)
    ax[fig_num].set_title(model_name + '  Feature Importances')
    ax[fig_num].set_xlabel('Importance')
    ax[fig_num].set_ylabel('Feature')
    return importances

# Load data from Azure Blob

In [10]:
data_df = get_raw_datasets(container, blob_name)
# data_df = data_df.loc[data_df['timestep_idx_local'] < 24*180, ].copy() # TODO: remove
data_df = data_df[[target_variable] + features + supplement_cols]
print(f"\nData size: {data_df.shape}")

# Drop gap flag hour = 1
data_df[data_df["gap_flag_hour"] == float(0)].reset_index(drop=True)
print(f"\nDrop gap-filled (hour) - Data size: {data_df.shape}")

# Encode categorical variables <-- TODO: Change to Dummy for RFR
# ref: https://xgboost.readthedocs.io/en/stable/tutorials/categorical.html
for col in categorical_cols:
    data_df[col] = data_df[col].astype('category')

print(f"\nData size: {data_df.shape}")
print(data_df.columns)

Data size: (4862712, 51)
Data Columns: Index(['GPP_NT_VUT_REF', 'site_id', 'timestep_idx_local',
       'timestep_idx_global', 'datetime', 'date', 'year', 'month', 'day',
       'hour', 'TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA',
       'PA_ERA', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'lat', 'long', 'koppen_sub', 'koppen_main', 'c3c4',
       'c4_percent', 'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN', 'CSIF-SIFdaily',
       'PET', 'Ts', 'ESACCI-sm', 'MODIS_LC', 'NDWI', 'Percent_Snow', 'Fpar',
       'Lai', 'LST_Day', 'LST_Night', 'MODIS_IGBP', 'MODIS_PFT',
       'gap_flag_hour', 'gap_flag_month'],
      dtype='object')
NA count: 0

Data size: (4862712, 42)

Drop gap-filled (hour) - Data size: (4862712, 42)

Data size: (4862712, 42)
Index(['GPP_NT_VUT_REF', 'month', 'day', 'hour', 'koppen_sub', 'koppen_main',
       'MODIS_PFT', 'MODIS_IGBP', 'MODIS_LC', 'TA_ERA', 'SW_IN_ERA',
       'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'EVI', 'NDV

# Prepare data with train=splits0,1,2 test=4

In [19]:
VAL_INDEX = 3
TEST_INDEX = 4
fold_data = []
train_df, val_df, test_df = get_splited_datasets(data_df, VAL_INDEX, TEST_INDEX)
train = split_to_X_y(train_df)
val   = split_to_X_y(val_df)
test   = split_to_X_y(test_df)
train.X, val.X, test.X = normalize_real_cols(int(1), realNum_cols, train.X, val.X, test.X)

print(f"train_y({train.y.shape}), train_X({train.X.shape})")
print(f"val_y({val.y.shape}), val_X({val.X.shape})")
fold_data.append({"train": train, "val": val})

Normalizing real features (29)
Saved scaler to /root/co2-flux-hourly-gpp-modeling/code/src/preprocessing/preproc_objects/scaler_cv1.joblib.
Train data size: (2824272, 37).
Val data size: (1056072, 37).
Test data size: (982368, 37).
train_y((2824272,)), train_X((2824272, 37))
val_y((1056072,)), val_X((1056072, 37))


In [20]:
## Train model
# Define model
best_params = {}
best_params['n_estimators'] = 20
best_params['max_depth'] = 9
best_params['eta'] = 0.2
model = XGBRegressor(**best_params, random_state=42,
                     tree_method="approx", enable_categorical=True,
                     importance_type='gain',
                     n_jobs=-1, verbosity=0)

model.fit(train.X, train.y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=True, eta=0.2, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=20, n_jobs=-1,
             num_parallel_tree=None, predictor=None, ...)

In [24]:
def model_eval(model, X, y):
    y_actuals = y
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y_actuals, y_pred))
    mae = mean_absolute_error(y_actuals, y_pred)
    r2 = r2_score(y_actuals, y_pred)
    loss_std = np.std(y_actuals - y_pred)
    print(f"  RMSE: {rmse:.5f}, MAE: {mae:.5f}, R2/NSE: {r2:.5f}, Loss SD: {loss_std:.5f}")

# Evaluate on Test
print("Val Set:")
model_eval(model, val.X, val.y)

Val Set:
  RMSE: 3.60672, MAE: 2.00776, R2/NSE: 0.74325, Loss SD: 3.60669


In [22]:
def model_eval(model, X, y):
    y_actuals = y
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y_actuals, y_pred))
    mae = mean_absolute_error(y_actuals, y_pred)
    r2 = r2_score(y_actuals, y_pred)
    loss_std = np.std(y_actuals - y_pred)
    print(f"  RMSE: {rmse:.5f}, MAE: {mae:.5f}, R2/NSE: {r2:.5f}, Loss SD: {loss_std:.5f}")

# Evaluate on Test
print("Test Set:")
model_eval(model, test.X, test.y)

Test Set:
  RMSE: 3.53272, MAE: 1.85541, R2/NSE: 0.69033, Loss SD: 3.52889


In [23]:
# Save
exp_dir = model_dir + os.sep + f"xgboost_full_features" 
if not (os.path.exists(exp_dir)):
    os.makedirs(exp_dir)
print(f"Experiment logs saved to {exp_dir}.")

# Save models
filesname = f"xgboost_full_features.pkl"
pickle.dump(model, open( exp_dir + os.sep + filesname, 'wb'))
print(f"  save model to {exp_dir + os.sep + filesname}.")

Experiment logs saved to /root/co2-flux-hourly-gpp-modeling/data/models/xgboost_full_features.
  save model to /root/co2-flux-hourly-gpp-modeling/data/models/xgboost_full_features/xgboost_full_features.pkl.
